# Share geoDB Collections

This notebook demonstrates how to (un)publish data collections in the xcube geoDB.

In [1]:
from xcube_geodb.core.geodb import GeoDBClient

##  Login from any machine

Install xcube geoDB with command:
  
 `conda install xcube_geodb -c conda-forge`


In [20]:
### uncomment if not in managed environment
#client_id=YourID
#client_secret=YourSecret
#geodb = GeoDBClient(client_id=client_id, client_secret=client_secret, auth_mode="client-credentials")

## Login in managed environment

The environment is prepared with your user credentials, so you simply can start the client.

In [ ]:
geodb = GeoDBClient()

## Get your user name

In [4]:
geodb.whoami

'geodb_ci_test_user'

## Create Collection

In [6]:
import geopandas

# Have a look at fiona feature schema
collections = {
        "land_use":
        {
            "crs": 3794,
            "properties":
            {
                "RABA_PID": "float",
                "RABA_ID": "float",
                "D_OD": "date"
            }
        }
    }


geodb.create_collections(collections, clear=True)

gdf = geopandas.read_file('data/sample/land_use.shp')
geodb.insert_into_collection('land_use', gdf.iloc[:100,:]) # minimizing rows to 100, if you are in EDC, you dont need to make the subset.

Processing rows from 0 to 100


100 rows inserted into land_use

## Publish a Collection to the World

In [7]:
geodb.list_my_grants()

Grants
0  No Grants

Please change the second positional kwargs to the geodb user you want to grant access to, if you are on EDC, go ahead and use 'geodb_test5' user:

In [8]:
geodb.grant_access_to_collection("land_use", "geodb_admin")

Access granted on land_use to geodb_admin

In [9]:
geodb.list_my_grants()

database table_name      grantee privileges
0  geodb_ci_test_user   land_use  geodb_admin     SELECT

## Accessing the Collection as a different User

Let's access the collection as a different user (a test user in this case), whose credentials were exported as an environment variable. You should now see a land_use collection.

In [10]:
import os

In [11]:
test_client_id = os.environ.get("TEST_CLIENT_ID")
test_client_secret = os.environ.get("TEST_CLIENT_SECRET")
test_client_geodb_api_server_url=os.environ.get("TEST_GEODB_API_SERVER_URL")

In [12]:
geodb = GeoDBClient(client_id=test_client_id, client_secret=test_client_secret, auth_mode="client-credentials", server_url=test_client_geodb_api_server_url)
geodb.whoami

'geodb_test5'

In [13]:
geodb.get_my_collections()

owner                       database  \
0  geodb_8c2d3fbe-f7a9-4492-8068-121e47e61a4f                        test_db   
1  geodb_8c2d3fbe-f7a9-4492-8068-121e47e61a4f                        test_db   
2               geodb_9bfgsdfg-453f-445b-a459  geodb_9bfgsdfg-453f-445b-a459   

        table_name  
0  test_default_db  
1    test_deleting  
2         land_use

## Revoke access

Let's go back to the original user.

In [14]:
geodb = GeoDBClient()
geodb.whoami


'geodb_ci_test_user'

In [15]:
geodb.list_my_grants()

database table_name      grantee privileges
0  geodb_ci_test_user   land_use  geodb_admin     SELECT

In [16]:
geodb.revoke_access_from_collection("land_use", 'geodb_admin')

Access revoked from geodb_ci_test_user on land_use

In [17]:
geodb.list_my_grants()

Grants
0  No Grants

## Finally going back to the initial user and deleting the collection:


In [20]:
geodb.drop_collection('land_use')

Collection ['geodb_ci_test_user_land_use'] deleted